In [ ]:
#| default_exp covariables

# covariables

> Construcción de covariables para analizar delitos

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import requests

In [ ]:
#| export
DATA_PATH = "datos/"
DOWNLOADS_PATH = "datos/descargas/"

## descarga_datos_covariables

In [ ]:
#| export
def descarga_datos_covariables():
    """Descarga los archivos necesarios qe son demasiado grandes para el repositorio.
    
        - geometrías de manzanas
    """
    covariables_url = "https://www.dropbox.com/s/s49lb476wpwu2p1/covariables.gpkg?dl=1"
    r = requests.get(covariables_url, allow_redirects=True)
    open(DOWNLOADS_PATH + 'covariables.gpkg', 'wb').write(r.content)

In [ ]:
descarga_datos_covariables()

## get_diccionario_censo

In [ ]:
#| export
def get_diccionario_censo():
    """Regresa un DataFrame con el diccionario de variables del censo."""
    dicionario = pd.read_csv("datos/diccionario_datos_ageb_urbana_09_cpv2020.csv", skiprows=3)
    diccionario = (dicionario
                   .drop(range(0,8))
                   .drop(columns='Núm.')
                   .reset_index(drop=True)
                   .rename({'Mnemónico':'Nombre del Campo'}, axis=1))
    return diccionario

In [ ]:
diccionario = get_diccionario_censo()
diccionario.head()

,Indicador,Descripción,Nombre del Campo,Rangos,Longitud
0,Población total,Total de personas que residen habitualmente en...,POBTOT,0...999999999,9
1,Población femenina,Total de mujeres que residen habitualmente en ...,POBFEM,0...999999999,9
2,Población masculina,Total de hombres que residen habitualmente en ...,POBMAS,0...999999999,9
3,Población de 0 a 2 años,Personas de 0 a 2 años de edad.,P_0A2,0…999999999,9
4,Población femenina de 0 a 2 años,Mujeres de 0 a 2 años de edad.,P_0A2_F,"0.,.999999999",9


## get_variables_censo

In [ ]:
#| export
def get_variables_censo():
    """Regresa un DataFrame con las variables del censo a nivel manzana."""
    df = pd.read_csv("datos/censo_manzanas.zip", dtype={'CVEGEO':str, 'colonia_cve': 'Int64'})
    return df

In [ ]:
censo = get_variables_censo()
censo.head()

,CVEGEO,AMBITO,TIPOMZA,POBTOT,POBFEM,POBMAS,P_0A2,P_0A2_F,P_0A2_M,P_3YMAS,...,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,colonia_cve,cuadrante_id
0,0901000010898031,Urbana,Típica,93.0,56.0,37.0,4.0,NaN,3.0,89.0,...,15.0,16.0,6.0,3.0,0.0,0.0,7.0,0.0,1119,17.0
1,0901000012269024,Urbana,Típica,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1082,14.0
2,0901000011472068,Urbana,Típica,124.0,66.0,58.0,3.0,3.0,0.0,121.0,...,25.0,22.0,9.0,8.0,0.0,NaN,7.0,0.0,1030,11.0
3,0901000011824024,Urbana,Típica,340.0,177.0,163.0,12.0,8.0,4.0,327.0,...,69.0,56.0,29.0,14.0,NaN,NaN,25.0,NaN,1135,110.0
4,0901000012377004,Urbana,Típica,82.0,41.0,41.0,NaN,0.0,NaN,80.0,...,13.0,13.0,6.0,3.0,0.0,0.0,9.0,0.0,1081,18.0


## imputa_faltantes_manzana

In [ ]:
#| export
def imputa_faltantes_manzana(censo, cols, metodo='ceros'):
    """ Regresa un df con los datos faltantes imputados a nivel manzana.
    
        params:
        
        cols: list: lista con las columnas en donde se deben imputar faltantes.
        metodo: método a usar para la imputación.
        NOTA: Por lo pronto sólo implementa dos métodos muy simples: llena con ceros 
        o aleatorio entre 0 y 3. En el futuro podríamos implementar mejores formas
    """
    if metodo == 'ceros':
        censo = censo.fillna(0)
    elif metodo == 'random':
        rand = pd.DataFrame(np.random.randint(0, 4, size=(censo.shape[0], len(cols))),
                            columns=cols, 
                            index=censo.index)
        censo.update(rand) 
    else:
        raise ValueError("imputacion debe ser ceros o random")
    return censo

In [ ]:
# Prueba funcionamiento normal
vars_pob = [v for v in diccionario['Nombre del Campo'].unique() 
            if (v.startswith('P') and v != 'PROM_HNV') ]
vars_viv = [v for v in diccionario['Nombre del Campo'].unique() if v.startswith('V')]
vars_viv.append('OCUPVIVPAR')
cols = vars_pob + vars_viv
imputados_ceros = imputa_faltantes_manzana(censo.head().copy(), cols)
imputados_rand = imputa_faltantes_manzana(censo.head().copy(), cols, metodo='random')
assert np.count_nonzero(imputados_rand.loc[:,cols].head().isna()) == 0
assert np.count_nonzero(imputados_ceros.loc[:,cols].head().isna()) == 0
# Prueba que arroje la excepción
# TODO

## agrega_en_unidades

In [ ]:
#| export 
def agrega_en_unidades(censo, diccionario,
                       agregacion        = 'colonias', 
                       imputacion        = 'ceros',
                       umbral_faltantes  = 0.5):
    """ Agrega las variables del censo en las unidades espaciales especificadas.
    
        params:
        agregacion: str: colonias/cuadrantes o nombre del campo
        imputacion: str: ceros/random. método para rellenar los datos faltantes. 
                             ceros llena con ceros, random con un aleatorio entre 0 y 3 
                             (faltantes por secreto)
        umbral_faltantes float: Porcentaje de datos faltantes en una manzana para 
                                considerarla en el análisis
                            
        NOTA: Las columnas PROM_HNV, GRAPROES(F/M) se pierden porque 
        no hay forma de calcularlas.
    """
    vars_pob = [v for v in diccionario['Nombre del Campo'].unique() 
                if (v.startswith('P') and v != 'PROM_HNV') ]
    vars_viv = [v for v in diccionario['Nombre del Campo'].unique() if v.startswith('V')]
    vars_viv.append('OCUPVIVPAR')
    if agregacion == 'colonias':
        columna_agrega = 'colonia_cve'
    elif agregacion == 'cuadrantes':
        columna_agrega = 'cuadrante_id'
    else:
        columna_agrega = agregacion
    assert columna_agrega in censo.columns
    censo.dropna(thresh=umbral_faltantes*(len(vars_pob) + len(vars_viv)), inplace=True)
    censo = imputa_faltantes_manzana(censo, vars_pob + vars_viv, metodo=imputacion)
    censo = censo[[columna_agrega] + vars_pob + vars_viv].groupby(columna_agrega).sum()
    # Calculamos las columnas que requieren trato espacial
    censo['PROM_OCUP'] = censo['OCUPVIVPAR'].div(censo['VIVPAR_HAB'])
    censo['PROM_OCUP_C'] = censo['OCUPVIVPAR'].div(censo['VPH_1CUART'] + 2*censo['VPH_2CUART'] + 3*censo['VPH_3YMASC'])
    return censo

In [ ]:
agregado = agrega_en_unidades(censo, diccionario, imputacion='random')
agregado = agrega_en_unidades(censo, diccionario)
agregado

,POBTOT,POBFEM,POBMAS,P_0A2,P_0A2_F,P_0A2_M,P_3YMAS,P_3YMAS_F,P_3YMAS_M,P_5YMAS,...,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,OCUPVIVPAR,PROM_OCUP_C
colonia_cve,,,,,,,,,,,,,,,,,,,,,
0,126.0,132.0,111.0,132.0,131.0,117.0,126.0,124.0,113.0,126.0,...,121.0,113.0,125.0,108.0,125.0,131.0,113.0,116.0,111.0,0.143782
1,81.0,79.0,79.0,88.0,93.0,79.0,78.0,97.0,65.0,90.0,...,79.0,91.0,93.0,86.0,88.0,86.0,90.0,87.0,85.0,0.177824
2,104.0,102.0,105.0,113.0,106.0,104.0,79.0,96.0,92.0,88.0,...,85.0,97.0,89.0,110.0,111.0,101.0,96.0,123.0,117.0,0.203478
3,19.0,25.0,23.0,22.0,28.0,23.0,20.0,25.0,22.0,20.0,...,24.0,24.0,15.0,19.0,26.0,24.0,24.0,22.0,25.0,0.183824
4,44.0,33.0,37.0,42.0,46.0,44.0,37.0,42.0,33.0,49.0,...,43.0,49.0,40.0,55.0,44.0,53.0,53.0,48.0,48.0,0.208696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,76.0,95.0,92.0,92.0,90.0,87.0,82.0,89.0,96.0,83.0,...,107.0,95.0,93.0,107.0,82.0,77.0,86.0,101.0,81.0,0.157588
1821,204.0,213.0,233.0,225.0,196.0,207.0,223.0,195.0,202.0,208.0,...,189.0,211.0,202.0,219.0,205.0,210.0,233.0,225.0,212.0,0.174486
1822,4.0,4.0,2.0,6.0,9.0,7.0,6.0,6.0,7.0,9.0,...,8.0,8.0,3.0,9.0,5.0,7.0,1.0,6.0,2.0,0.090909


## censo_a_tasas

In [ ]:
#| export
def censo_a_tasas(censo, diccionario, umbral_faltantes=0.5):
    """Convierte las variables del censo a tasas en la agregación seleccionada.

    Para las variables de población divide por población total.
    Para Hogares divide por TOTHOG.
    Para viviendas divide por el total de viviendas particulares 
     habitadas (VIVPAR_HAB)
    
    params:
    umbral_faltantes float: Porcentaje de datos faltantes en una unidad para 
    considerarla en el análisis
    """
    pob_col = 'POBTOT'
    hog_col = 'TOTHOG'
    viv_col = 'VIVPAR_HAB'
    vars_pob = [v for v in diccionario['Nombre del Campo'].unique() if v.startswith('P')]
    eliminar = ['POBTOT', 'PROM_HNV']
    vars_pob = [v for v in vars_pob if (v not in eliminar)]
    vars_viv = [v for v in diccionario['Nombre del Campo'].unique() 
                if (v.startswith('V') and v != 'VIVPAR_HAB')]
    censo[vars_pob] = censo[vars_pob].div(censo[pob_col], axis=0)
    censo[vars_viv] = censo[vars_viv].div(censo[viv_col], axis=0)
    censo.dropna(thresh=umbral_faltantes*(len(vars_pob) + len(vars_viv)), inplace=True)
    return censo

In [ ]:
agregado_tasas = censo_a_tasas(agregado, diccionario)
manzanas_tasas = censo_a_tasas(imputados_ceros, diccionario)

## get_uso_de_suelo

In [ ]:
#| export
def get_uso_de_suelo():
    """Regresa un DataFrame con las variables de uso de suelo a nivel manzana."""
    df = pd.read_csv("datos/usos_suelo.csv", dtype={'CVEGEO':str, 
                                                    'colonia_cve': 'Int64',
                                                    'cuadrante_id':str})
    return df

In [ ]:
uso_suelo = get_uso_de_suelo()
uso_suelo

,CVEGEO,colonia_cve,cuadrante_id,Industria,Comercio,Servicios
0,0901000010898031,1119,017,1,4,5
1,0901000012269024,1082,014,0,0,0
2,0901000011472068,1030,011,1,1,0
3,0901000011824024,1135,0110,0,3,2
4,0901000012377004,1081,018,0,0,0
...,...,...,...,...,...,...
66379,0900700015376020,1442,019,0,0,0
66380,0900700015376021,1442,019,1,4,4
66381,0900700013045056,1419,0113,0,0,0
66382,0900700013045032,1419,0113,2,13,14


## agrega_uso_suelo

In [ ]:
#| export
def agrega_uso_suelo(usos, unidades='colonias'):
    """ Regresa un DataFrame con los usos agregados en las unidades espaciales.
    
        Además calcula la intensidad y la entropía para cada unidad.
        
        params:
        usos: DataFrame: lo que sale de `get_uso_de_suelo()`
        unidades: str: colonias/cuadrantes
        
        NOTA: eventualmente debe recibir unidades arbitrarias.
    """
    if unidades == 'colonias':
        columna_agrega = 'colonia_cve'
    elif unidades == 'cuadrantes':
        columna_agrega = 'cuadrante_id'
    else:
        raise ValueError("unidades debe ser 'colonias' o 'cuadrantes'")
    usos = usos.groupby(columna_agrega).sum()
    usos['Intensidad'] = usos.sum(axis=1)
    usos['Entropía'] = (np.log(usos[['Industria', 'Comercio', 'Servicios']]
                               .div(usos['Intensidad'], axis=0))
                        .sum(axis=1) / np.log(3))
    return usos

In [ ]:
us = agrega_uso_suelo(uso_suelo, unidades='colonias')
assert len(set(['Intensidad', 'Entropía']).intersection(set(us.columns))) == 2
us = agrega_uso_suelo(uso_suelo, unidades='cuadrantes')
assert len(set(['Intensidad', 'Entropía']).intersection(set(us.columns))) == 2
# TODO probar que arroja la excepción

## IndicePCA

In [ ]:
#| export
class IndicePCA(object):
    """ Clase contenedora para los índices basados en PCA.
    
        Args:
            covariables: DataFrame: del mismo tipo que `agrega_en_unidades`. El índice
            del DataFrame debe ser el id de la unidad de agregación
            vars_indice: list: la lista de las columnas con las qeu calculamos el índice
    """
    def __init__(self, covariables, vars_indice):
        self.datos = covariables
        self.vars_indice = vars_indice
        self.varianza_explicada = None
        self.indice = None
        
    def calcula_indice(self):
        """Calcula el índice y lo guarda en self.indice"""
        pca = PCA(n_components=1, svd_solver='full', random_state=1)
        indicadores = self.datos.replace([np.inf, -np.inf], np.nan)
        indicadores = indicadores[self.vars_indice].dropna()
        X = StandardScaler().fit_transform(indicadores.values)
        pca.fit(X)
        self.varianza_explicada = pca.explained_variance_ratio_
        indice = pca.fit_transform(X)[:,:1]
        id_var = indicadores.index.name
        df = indicadores.reset_index()[[id_var]]
        # df['Índice'] = abs(indice)
        df['Índice'] = indice
        self.indice = df

In [ ]:
diccionario = get_diccionario_censo()
censo = get_variables_censo()
agregado = agrega_en_unidades(censo, diccionario, imputacion='random')
agregado = censo_a_tasas(agregado, diccionario)
vars_indice = ['P5_HLI', 'POB_AFRO', 'PCON_DISC', 'P3A5_NOA', 
               'P6A11_NOA', 'P12A14NOA', 'P15YM_AN', 'PSINDER', 'PDESOCUP']
# indice = get_indice_pca(agregado, vars_indice)
# indice
indice = IndicePCA(agregado, vars_indice)
indice.calcula_indice()
print(indice.varianza_explicada)

[0.68589544]
